In [75]:
%pylab inline
import sys
sys.path.append('../src/')
from utils import *

Populating the interactive namespace from numpy and matplotlib


/home/isaac/anaconda2/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['test']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [29]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
comb = pd.concat((train.drop('SalePrice', axis=1), test)).set_index('Id')

In [68]:
cols = list(comb.columns)
list(comb.columns[comb.isnull().any(axis=0)])

['MSZoning',
 'LotFrontage',
 'Alley',
 'Utilities',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType']

In [ ]:
tofill = {}

In [177]:
tofill

{(27, 'BsmtExposure'): 'No',
 (332, 'BsmtfinType2'): 'Unf',
 (333, 'BsmtfinType2'): 'Unf',
 (580, 'BsmtCond'): 'TA',
 (725, 'BsmtCond'): 'TA',
 (888, 'BsmtExposure'): 'No',
 (948, 'BsmtExposure'): 'No',
 (949, 'BsmtExposure'): 'No',
 (1064, 'BsmtCond'): 'TA',
 (1488, 'BsmtExposure'): 'No',
 (1556, 'KitchenQual'): 'TA',
 (1916, 'MSZoning'): 'RL',
 (2041, 'BsmtCond'): 'TA',
 (2186, 'BsmtCond'): 'TA',
 (2217, 'MSZoning'): 'RL',
 (2218, 'BsmtQual'): 'TA',
 (2219, 'BsmtQual'): 'TA',
 (2251, 'MSZoning'): 'RL',
 (2349, 'BsmtExposure'): 'No',
 (2525, 'BsmtCond'): 'TA',
 (2905, 'MSZoning'): 'RL'}

## Exterior

In [193]:
comb[['Exterior1st', 'MSZoning', 'MSSubClass', 'YearBuilt']][comb.Exterior1st.isnull()]

,Exterior1st,MSZoning,MSSubClass,YearBuilt
Id,,,,
2152,NaN,RL,30,1940


In [195]:
comb[(comb.YearBuilt > 1935) & (comb.YearBuilt < 1945)].groupby('Exterior1st').YearBuilt.count()

Exterior1st
AsbShng     3
BrkFace     3
CemntBd     1
MetalSd    40
Plywood     1
Stucco      2
VinylSd    16
Wd Sdng    46
WdShing     5
Name: YearBuilt, dtype: int64

In [199]:
comb[['Exterior1st', 'Exterior2nd', 'MSZoning', 'MSSubClass', 'YearBuilt']][comb.Exterior2nd.isnull()]

,Exterior1st,Exterior2nd,MSZoning,MSSubClass,YearBuilt
Id,,,,,
2152,NaN,NaN,RL,30,1940


In [200]:
comb[(comb.YearBuilt > 1935) & (comb.YearBuilt < 1945)].groupby('Exterior2nd').YearBuilt.count()

Exterior2nd
AsbShng     3
BrkFace     2
CmentBd     1
HdBoard     1
MetalSd    40
Plywood     2
Stucco      2
VinylSd    17
Wd Sdng    43
Wd Shng     6
Name: YearBuilt, dtype: int64

In [201]:
tofill[(2152, 'Exterior1st')] = 'Wd Sdng'
tofill[(2152, 'Exterior2nd')] = 'MetalSd'

## Zoning

In [142]:
[x for x in comb.columns if 'Zon' in x]

['MSZoning']

In [143]:
comb.MSZoning.isnull().sum()

4

In [149]:
comb.groupby('MSZoning').YrSold.count()

MSZoning
C (all)      25
FV          139
RH           26
RL         2265
RM          460
Name: YrSold, dtype: int64

In [150]:
comb.groupby('MSZoning').LotArea.agg(['mean', 'median', 'count'])

,mean,median,count
MSZoning,,,
C (all),8320.720000,8500,25
FV,6549.359712,7379,139
RH,7506.461538,7168,26
RL,11169.471965,10000,2265
RM,6400.239130,6090,460


In [155]:
comb[['MSZoning', 'MSSubClass']][comb.MSZoning.isnull()]

,MSZoning,MSSubClass
Id,,
1916,NaN,30
2217,NaN,20
2251,NaN,70
2905,NaN,20


In [163]:
comb[comb.MSSubClass == 70].groupby('MSZoning').YrSold.count()

MSZoning
C (all)     4
RH          3
RL         57
RM         63
Name: YrSold, dtype: int64

In [176]:
tofill[(1916, 'MSZoning')] = 'RL'
tofill[(2217, 'MSZoning')] = 'RL'
tofill[(2251, 'MSZoning')] = 'RL'
tofill[(2905, 'MSZoning')] = 'RL'

## Fireplace

In [127]:
[x for x in comb.columns if 'Fire' in x]

['Fireplaces', 'FireplaceQu']

In [128]:
comb.FireplaceQu.isnull().sum()

1420

In [129]:
(comb.FireplaceQu.isnull() & (comb.Fireplaces != 0)).sum()

0

In [131]:
((comb.Fireplaces == 0) & ~comb.FireplaceQu.isnull()).sum()

0

## Fence

In [161]:
[x for x in comb.columns if 'Fence' in x or 'Fnc' in x]

['Fence']

In [126]:
comb.Fence.isnull().sum()

2348

In [125]:
comb.groupby('Fence').YearBuilt.count()

Fence
GdPrv    118
GdWo     112
MnPrv    329
MnWw      12
Name: YearBuilt, dtype: int64

## Kitchen

In [174]:
comb[comb.KitchenQual.isnull()]

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,


In [119]:
comb.groupby('KitchenQual').KitchenAbvGr.count()

KitchenQual
Ex     205
Fa      70
Gd    1151
TA    1492
Name: KitchenAbvGr, dtype: int64

In [120]:
comb.groupby('KitchenQual')['YearRemodAdd'].agg(['mean', 'median', 'count'])

,mean,median,count
KitchenQual,,,
Ex,2003.009756,2006,205
Fa,1962.114286,1950,70
Gd,1997.775847,2003,1151
TA,1972.327748,1970,1492


In [175]:
tofill[(1556, 'KitchenQual')] = 'TA'

## Electrical

In [112]:
comb.groupby('Electrical')['YearBuilt'].agg(['mean', 'median', 'count'])

,mean,median,count
Electrical,,,
FuseA,1937.680851,1943.5,188
FuseF,1933.360000,1931.0,50
FuseP,1929.625000,1925.0,8
Mix,1920.000000,1920.0,1
SBrkr,1974.521528,1977.0,2671


In [202]:
comb[['Electrical']][comb.Electrical.isnull()]

,Electrical
Id,
1380,NaN


In [113]:
comb[comb.Electrical.isnull()].iloc[0].Electrical = 'SBrkr'

## Masonry Columns

In [97]:
mas = ['MasVnrType', 'MasVnrArea']
[comb[m].isnull().sum() for m in mas]

[24, 23]

In [180]:
comb[['MasVnrType', 'MasVnrArea']][nomas]

,MasVnrType,MasVnrArea
Id,,
235,None,0.0
530,None,0.0
651,None,0.0
937,None,0.0
974,None,0.0
978,None,0.0
1244,None,0.0
1279,None,0.0
1692,None,0.0


In [99]:
nomas = comb.MasVnrType.isnull() & comb.MasVnrArea.isnull()
comb.MasVnrType[nomas] = 'None'
comb.MasVnrArea[nomas] = 0.0

/home/isaac/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/isaac/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [102]:
comb[comb['MasVnrArea'] == 0].groupby('MasVnrType').count()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
MasVnrType,,,,,,,,,,,,,,,,,,,,,
BrkFace,2,2,2,2,2,0,2,2,2,2,...,2,2,0,0,0,2,2,2,2,2
None,1758,1754,1500,1758,1758,171,1758,1758,1756,1758,...,1758,1758,5,400,75,1758,1758,1758,1758,1758
Stone,1,1,1,1,1,0,1,1,1,1,...,1,1,0,0,0,1,1,1,1,1


In [103]:
comb[comb.MasVnrArea == 0].MasVnrType = 'None'

In [82]:
comb[(comb.MasVnrArea == 0) & ~comb.MasVnrType.isnull() & ~(comb.MasVnrType=='None')].MasVnrType

Id
689     BrkFace
1242      Stone
2320    BrkFace
Name: MasVnrType, dtype: object

In [84]:
comb.groupby('MasVnrType').YearBuilt.median()

MasVnrType
BrkCmn     1961
BrkFace    1990
None       1962
Stone      2006
Name: YearBuilt, dtype: int64

In [167]:
comb[comb.MasVnrType.isnull() & ~comb.MasVnrArea.isnull()][['MasVnrArea', 'MasVnrType']]

,MasVnrArea,MasVnrType
Id,,
2611,198.0,NaN


In [182]:
tofill[(2611, 'MasVnrType')] = 'BrkCmn'

## Basement Columns

In [31]:
bsmt = [x for x in cols if 'Bsmt' in x]

In [32]:
train[bsmt].isnull().any(axis=0)

BsmtQual         True
BsmtCond         True
BsmtExposure     True
BsmtFinType1     True
BsmtFinSF1      False
BsmtFinType2     True
BsmtFinSF2      False
BsmtUnfSF       False
TotalBsmtSF     False
BsmtFullBath    False
BsmtHalfBath    False
dtype: bool

In [33]:
bsmtnums = ['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath']

In [34]:
comb[bsmtnums] = comb[bsmtnums].fillna(0)

In [35]:
bsmtna = list(comb[bsmt].columns[comb[bsmt].isnull().any(axis=0)])

In [36]:
bsmtna

['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']

In [87]:
allna = comb[bsmtna].isnull().all(axis=1)
anyna = comb[bsmtna].isnull().any(axis=1)
allze = (comb[bsmtnums] == 0).all(axis=1)
comb[bsmtna][anyna & ~allna]

,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2
Id,,,,,
333,Gd,TA,No,GLQ,NaN
949,Gd,TA,NaN,Unf,Unf
1488,Gd,TA,NaN,Unf,Unf
2041,Gd,NaN,Mn,GLQ,Rec
2186,TA,NaN,No,BLQ,Unf
2218,NaN,Fa,No,Unf,Unf
2219,NaN,TA,No,Unf,Unf
2349,Gd,TA,NaN,Unf,Unf
2525,TA,NaN,Av,ALQ,Unf


In [23]:
# Row 2349 should have BsmtExposure = 'No'
comb[(comb.BsmtCond == 'TA') & (comb.BsmtFinType2 =='Unf') &
     (comb.BsmtQual=='Gd') & (comb.BsmtFinType1 == 'Unf')] \
    .groupby('BsmtExposure').Id.count()

BsmtExposure
Av     58
Gd     10
Mn     22
No    255
Name: Id, dtype: int64

In [15]:
# BsmtExposure should probably be 'No' in rows 949 and 1488
comb[(comb.BsmtQual == 'Gd') & (comb.BsmtCond=='TA') &
     (comb.BsmtFinType2=='Unf') & (comb.BsmtFinType1 == 'Unf')] \
    .groupby('BsmtExposure').Id.count()

BsmtExposure
Av     58
Gd     10
Mn     22
No    255
Name: Id, dtype: int64

In [22]:
# Row 2219 should have BsmtQual = 'TA'?
comb[(comb.BsmtCond == 'TA') & (comb.BsmtFinType2 =='Unf') &
     (comb.BsmtExposure=='No') & (comb.BsmtFinType1 == 'Unf')] \
    .groupby('BsmtQual').Id.count()

BsmtQual
Ex     20
Fa     38
Gd    255
TA    304
Name: Id, dtype: int64

In [21]:
# Row 2218 should have BsmtQual = 'TA'?
comb[(comb.BsmtCond == 'Fa') & (comb.BsmtFinType2 =='Unf') &
     (comb.BsmtExposure=='No') & (comb.BsmtFinType1 == 'Unf')] \
    .groupby('BsmtQual').Id.count()

BsmtQual
Fa    13
Gd     2
TA    31
Name: Id, dtype: int64

In [20]:
# Row 2525 should have BsmtCond = 'TA'
comb[(comb.BsmtQual == 'TA') & (comb.BsmtFinType2 =='Unf') &
     (comb.BsmtExposure=='Av') & (comb.BsmtFinType1 == 'ALQ')] \
    .groupby('BsmtCond').Id.count()

BsmtCond
Gd     1
TA    28
Name: Id, dtype: int64

In [17]:
# Row 2186 should have BsmtCond = 'TA'
comb[(comb.BsmtQual == 'TA') & (comb.BsmtFinType2 =='Unf') &
     (comb.BsmtExposure=='No') & (comb.BsmtFinType1 == 'BLQ')] \
    .groupby('BsmtCond').Id.count()

BsmtCond
Fa      7
Gd      2
TA    121
Name: Id, dtype: int64

In [16]:
# Row 2041 should have BsmtCond = 'TA'
comb[(comb.BsmtQual == 'Gd') & (comb.BsmtFinType2 =='Rec') &
     (comb.BsmtExposure=='Mn') & (comb.BsmtFinType1 == 'GLQ')] \
    .groupby('BsmtCond').Id.count()

BsmtCond
TA    1
Name: Id, dtype: int64

In [14]:
# Clearly BsmtFinType2 should be 'Unf' in row 333
comb[(comb.BsmtQual == 'Gd') & (comb.BsmtCond=='TA') &
     (comb.BsmtExposure=='No') & (comb.BsmtFinType1 == 'GLQ')] \
    .groupby('BsmtFinType2').Id.count()

BsmtFinType2
ALQ      6
BLQ      1
LwQ      2
Rec      3
Unf    242
Name: Id, dtype: int64

In [96]:
# To summarize so far:
# (Id, col): value
tofill.update({
    (949, 'BsmtExposure'): 'No',
    (1488,'BsmtExposure'): 'No',
    (333, 'BsmtfinType2'): 'Unf',
    (2041,'BsmtCond'):     'TA',
    (2186,'BsmtCond'):     'TA',
    (2525,'BsmtCond'):     'TA',
    (2349,'BsmtExposure'): 'No',
})

In [45]:
# Still unsure: BsmtQual in 2218 and 2219
comb.ix[[2218,2219]][bsmtnums]

,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,BsmtFullBath,BsmtHalfBath
Id,,,,,,
2218,0.0,0.0,173.0,173.0,0.0,0.0
2219,0.0,0.0,356.0,356.0,0.0,0.0


In [64]:
# 'Fa' basements tend to be smaller than 'TA' basements, and these basements are small.
# However, 'TA' basements are more common in general, and in other situations with similar
# categoricals. Best bet is probably to label them 'TA'.
comb[(comb.BsmtUnfSF == comb.TotalBsmtSF)] \
    .groupby('BsmtQual').TotalBsmtSF.agg(['count', 'mean'])

,count,mean
BsmtQual,,
Ex,56,1443.553571
Fa,60,655.500000
Gd,364,1048.774725
TA,369,877.994580


In [89]:
comb[(comb.BsmtUnfSF == comb.BsmtUnfSF)] \
    .groupby('BsmtQual').TotalBsmtSF.agg(['count', 'mean'])

,count,mean
BsmtQual,,
Ex,258,1648.317829
Fa,88,719.909091
Gd,1209,1120.267990
TA,1283,955.211224


In [65]:
tofill[(2218, 'BsmtQual')] = 'TA'
tofill[(2219, 'BsmtQual')] = 'TA'